In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, load_from_disk
from tqdm import tqdm
import json

c:\Users\linxi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model_path = './tinybert-filter'
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = BertForSequenceClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

with open('./jsonl/all_data_to_bert.jsonl', 'r', encoding='utf-8') as f:
    input_texts = [json.loads(line) for line in f]

scored_data = []
for row in tqdm(input_texts, desc="Labelling"):
    text = row['text']
    index = row['index']
    dataset = row['dataset']
    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    ).to(device)
    with torch.no_grad():
        output = model(**inputs)
        label = output.logits.argmax(axis=-1).item()
        scored_data.append({
            'index': index,
            'label': label,
            'dataset': dataset,
            'text': text,
        })
    


Labelling: 100%|██████████| 225993/225993 [37:21<00:00, 100.81it/s]


In [13]:
filtered_data = []
with open('./jsonl/labelled_data_unfiltered.jsonl', 'w', encoding='utf-8') as f:
    for line in scored_data:
        if line['label'] == 1:
            filtered_data.append(line)
        f.write(json.dumps(line) + "\n")

filtered_data = Dataset.from_list(filtered_data)
filtered_data.save_to_disk('labelled_data_(filtered)')
filtered_data

Saving the dataset (1/1 shards): 100%|██████████| 129311/129311 [00:00<00:00, 828105.94 examples/s]


Dataset({
    features: ['index', 'label', 'dataset', 'text'],
    num_rows: 129311
})

In [18]:
ds_origin = load_from_disk('all_data')

filtered_data = load_from_disk('labelled_data_(filtered)')
indices = filtered_data['index']
ds_filtered = ds_origin.select(indices)
ds_filtered
ds_origin

Dataset({
    features: ['type', 'query', 'original_question', 'response', 'index', 'source', 'conversation', 'instruction', 'input', 'output', 'prompt', 'problem', 'solution', 'answer', 'problem_type', 'question_type', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages', 'turn_1', 'feedback_1', 'turn_2', 'feedback_2', 'dataset'],
    num_rows: 225993
})

In [ ]:
ds_formated = []
for sample in tqdm(ds_filtered, desc='Formatting'):
    if sample['query']:
        ds_formated.append({
            'data_type': sample['dataset'],
            'conversation':[{
                'from': 'human',
                'value': sample['query']
            }, {
                'from': 'gpt',
                'value': sample['response']
            }]
        })
    elif sample['conversation']:
        conversation = []
        for dialogue in sample['conversation']:
            conversation.append({
                'from': 'human',
                'value': dialogue['input']
            })
            conversation.append({
                'from': 'gpt',
                'value': dialogue['output']
            })
        ds_formated.append({
            'data_type': sample['dataset'],
            'conversation': conversation
        })
    elif sample['instruction']:
        prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
        ds_formated.append({
            'data_type': sample['dataset'],
            'conversation': [{
                'from': 'human',
                'value': prompt + "###Instruction:\n" + sample['instruction'] + "###Input:\n" + sample['input']
            }, {
                'from': 'gpt',
                'value': sample['output']
            }]
        })
    elif sample['problem']:
        ds_formated.append({
            'data_type': sample['dataset'],
            'conversation': [{
                'from': 'human',
                'value': sample['problem']
            }, {
                'from': 'gpt',
                'value': sample['solution']
            }]
        })
    elif sample['turn_1']:
        if sample['turn_2']:
            ds_formated.append({
                'data_type': sample['dataset'],
                'conversation': [{
                    'from': 'human',
                    'value': sample['prompt']
                }, {
                    'from': 'gpt',
                    'value': sample['turn_1']
                }, {
                    'from': 'human',
                    'value': sample['feedback_1']
                }, {
                    'from': 'gpt',
                    'value': 'turn_2'
                }]
            })
        else:
            ds_formated.append({
                'data_type': sample['dataset'],
                'conversation': [{
                    'from': 'human',
                    'value': sample['prompt']
                }, {
                    'from': 'gpt',
                    'value': sample['turn_1']
                }]
            })

ds_formated = Dataset.from_list(ds_formated)
ds_formated

Formatting: 100%|██████████| 129311/129311 [00:24<00:00, 5247.59it/s]


Dataset({
    features: ['data_type', 'conversation'],
    num_rows: 129311
})

In [21]:
ds_formated.save_to_disk('SFTdata')

Saving the dataset (1/1 shards): 100%|██████████| 129311/129311 [00:00<00:00, 505912.57 examples/s]
